Collect Betting Datas with Betmonitor

Get the required Libraries

In [168]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import win32com.client as win32
from datetime import datetime
import os
from datetime import datetime

Code

In [173]:
nb1_link = "https://www.betmonitor.com/odds-comparison/football/hungary-nb-i/10000110"
bl_link = "https://www.betmonitor.com/odds-comparison/football/uefa-champions-league/10000315"
el_link = "https://www.betmonitor.com/odds-comparison/football/uefa-europa-league/10000314"
nl_link = "https://www.betmonitor.com/odds-comparison/football/uefa-nations-league/10010255"
eb_sel_link = "https://www.betmonitor.com/odds-comparison/football/uefa-european-championship-qualification/10024064"
premier_leauge_link = "https://www.betmonitor.com/odds-comparison/football/england-premier-league/10000070"
serie_a_link = "https://www.betmonitor.com/odds-comparison/football/italy-serie-a/10000124"
ligue_1_link = "https://www.betmonitor.com/odds-comparison/football/france-ligue-1/10000080"
bundesliga_link = "https://www.betmonitor.com/odds-comparison/football/germany-bundesliga/10000090"
la_liga_link = "https://www.betmonitor.com/odds-comparison/football/spain-la-liga/10000184"

leauge_links = []
leauge_links.extend([
    nb1_link,
    bl_link,
    el_link,
    nl_link,
    eb_sel_link,
    premier_leauge_link,
    serie_a_link,
    ligue_1_link,
    bundesliga_link,
    la_liga_link
])

#Build the dataframe
OddsTable = pd.DataFrame(columns=["Leauge", "Teams", "Bookie", "HomeOdds", "DrawOdds", "AwayOdds", "Payout"])

#Get the links for matches

for leauge_url in leauge_links[8:]:

    options = Options()
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    # Set the chrome driver
    driver = webdriver.Chrome("YOUR_PATH")

    # Pings the specified url
    driver.get(leauge_url)

    # Sleep time to wait for 3 seconds to wait for page load
    time.sleep(3)

    # Gets the page source
    pg_leauge = driver.page_source

    soup_league = BeautifulSoup(pg_leauge, 'lxml')
    data_league = soup_league.find("div", id="league").find("div", id="content")
    search_league = data_league.find_all("div", class_="league-event-new")

    leauge_links = []

    #Get the match links
    for element_league in search_league:
        try:
            link = element_league.find("div", class_="info").find("div", class_="teams").find("a")["href"]
            link = f"https://www.betmonitor.com/{link}"
            leauge_links.append(link)
        except AttributeError:
            None

#Get the datas

    for url in leauge_links:

        # open the chrome driver
        driver = webdriver.Chrome("YOUR_PATH")

        # pings the specified url
        driver.get(url)

        # sleep time to wait for t seconds to wait for page load
        # replace 3 with any int value (int value in seconds)
        time.sleep(3)

        # gets the page source
        pg = driver.page_source

        #Build the Loop
        soup = BeautifulSoup(pg, 'lxml')
        data = soup.find("div", id="event").find("div", id="content").find("div", class_="odds").find("div", class_="bettype-cont items-3").find("div", class_="bookmakers-cont").find("div", class_="bookmakers").find("div", class_="bookmakers-sort")
        search = data.find_all("div", class_="bookie-cont sortable")

        #Get the default Datas
        Teams = soup.find("div", id="event").find("div", id="content").find("div", class_="event-header-container").find("div", class_="event-header").find("div", class_="info").find("div", class_="teams has-logo").text.strip()
        Leauge = soup.find("div", id="event").find("div", id="content").find("div", class_="event-header-container").find("div", class_="event-header").find("div", class_="info").find("div", class_="league").find("a", class_="dark").text
        Leauge = Leauge.split(" · ")
        Leauge = Leauge[-1:]
        Leauge = Leauge[0]

        #Data collecting and build into the dataframe
        for element in search:
            Bookie = element.find("div", class_="bookie").text
            
            bettype_q1 = element.find("div", class_=["bettype q1", "bettype q1 red"])
            if bettype_q1 is not None:
                Home_Odds = bettype_q1.text
                Home_Odds = Home_Odds[:4].replace(".", ",")

            bettype_qx = element.find("div", class_=["bettype qx", "bettype qx red"])
            if bettype_qx is not None:
                Draw_Odds = bettype_qx.text
                Draw_Odds = Draw_Odds[:4].replace(".", ",")

            bettype_q2 = element.find("div", class_=["bettype q2", "bettype q2 red"])
            if bettype_q2 is not None:
                Away_Odds = bettype_q2.text
                Away_Odds = Away_Odds[:4].replace(".", ",")
            
            Payout = element.find("div", class_="payout").text.replace(".", ",")


            OddsTable_Temp = pd.DataFrame([[Leauge, Teams, Bookie, Home_Odds, Draw_Odds, Away_Odds, Payout]], columns=["Leauge", "Teams", "Bookie", "HomeOdds", "DrawOdds", "AwayOdds", 'Payout'])
            OddsTable = pd.concat([OddsTable, OddsTable_Temp], ignore_index=True)

driver.quit()

#Create the email object
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

#Get date
now = datetime.now()  # get the current date and time
date_text = now.strftime("%Y-%m-%d_%H_%M")  # format the date as text with hour and minute


#Convert the DataFrame to an Excel file
excel_file = pd.ExcelWriter(f"Oddsok{date_text}.xlsx", engine="xlsxwriter")
OddsTable.to_excel(excel_file, index=False)
excel_file.save()

#Get the current working directory
cwd = os.getcwd()

#Combine the current working directory with the file name to get the file path
file_path = os.path.join(cwd, f"Oddsok{date_text}.xlsx")

#Set the email parameters
mail.Subject = f'Oddsok {date_text}'
mail.To = "YOUR_EMAIL_ADDRESS"
mail.HTMLBody = r"""
<p>YOUR MESSAGE<p>
"""
mail.Attachments.Add(file_path)

#Send the email
mail.Send()

C:\Users\francuza\AppData\Local\Temp\ipykernel_18840\979332317.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/francuza/Downloads/chromedriver_win32/chromedriver.exe")
C:\Users\francuza\AppData\Local\Temp\ipykernel_18840\979332317.py:69: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/francuza/Downloads/chromedriver_win32/chromedriver.exe")
C:\Users\francuza\AppData\Local\Temp\ipykernel_18840\979332317.py:69: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/francuza/Downloads/chromedriver_win32/chromedriver.exe")
C:\Users\francuza\AppData\Local\Temp\ipykernel_18840\979332317.py:69: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/francuza/Downloads/chromedriver_win32/chr

In [172]:
#Create the email object
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

#Get date
now = datetime.now()  # get the current date and time
date_text = now.strftime("%Y-%m-%d_%H_%M")  # format the date as text with hour and minute


#Convert the DataFrame to an Excel file
excel_file = pd.ExcelWriter(f"Oddsok{date_text}.xlsx", engine="xlsxwriter")
OddsTable.to_excel(excel_file, index=False)
excel_file.save()

#Get the current working directory
cwd = os.getcwd()

#Combine the current working directory with the file name to get the file path
file_path = os.path.join(cwd, f"Oddsok{date_text}.xlsx")

#Set the email parameters
mail.Subject = f'Oddsok {date_text}'
mail.To = "Francuz.Adam@szerencsejatek.hu"
mail.HTMLBody = r"""
<p>Hello!,<p>
<p>Ez egy teszt email.<p>
<p>Best regards,<p>
<p>Ádám<p>
"""
mail.Attachments.Add(file_path)

#Send the email
mail.Send()